In [27]:
#輸出csv字串
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai
import os
genai.configure(api_key=os.environ['GEMINI_API_KEY'])
# response = requests.get('https://www.merit-times.com.tw/NewsPage.aspx?unid=903451')
response = requests.get('https://rate.bot.com.tw/xrt?Lang=zh-TW')
soup = BeautifulSoup(response.text,"html.parser")
body_content = soup.body
for script_or_style in body_content(['script','style']):
    script_or_style.extract()

table_content = body_content.find(title='牌告匯率')
table_lines = table_content.get_text(separator='\n')
cleaned_content = "\n".join(
    [line.strip() for line in table_lines.splitlines() if line.strip()]
    )


model = genai.GenerativeModel(
    "gemini-2.0-flash-exp",
    system_instruction='''
    1. 你的任務是取出指定的內容,並輸出成csv的格式
    2. 只要輸出csv格式的字串,不要有多餘的文字

    請依照下面的指示:    
    ### 如果找到下面的樣本:
    ```
    澳幣 (AUD)
    澳幣 (AUD)
    20.15
    20.93
    20.365
    20.71
    ```

    ### 輸出的csv格式:
    1. 第1欄:有欄位名稱
    2. 欄位名稱
    - 幣別:str國幣的名稱
    - 幣別代碼:str國家的代碼
    - `現金匯率(本行買入)`:float,無法轉換請用null
    - `現金匯率(本行賣出)`:float,無法轉換請用null
    - `即期匯率(本行買入)`:float,無法轉換請用null
    - `即期匯率(本行賣出)`:float,無法轉換請用null

    ### 輸出的樣本
    ```
    幣別,幣別代碼,現金匯率(本行買入),現金匯率(本行賣出),即期匯率(本行買入),即期匯率(本行賣出)
    澳幣,AUD,20.15,20.93,20.365,20.71
    ```  
    '''
)

response = model.generate_content(cleaned_content)
result_text = response.text
print(result_text)




幣別,幣別代碼,現金匯率(本行買入),現金匯率(本行賣出),即期匯率(本行買入),即期匯率(本行賣出)
美金,USD,32.28,32.95,32.605,32.755
港幣,HKD,4.04,4.244,4.161,4.231
英鎊,GBP,39.46,41.58,40.355,40.985
澳幣,AUD,20.03,20.81,20.245,20.59
加拿大幣,CAD,22.19,23.1,22.52,22.85
新加坡幣,SGD,23.6,24.51,24.07,24.29
瑞士法郎,CHF,35.25,36.45,35.86,36.25
日圓,JPY,0.2011,0.2139,0.2079,0.2129
南非幣,ZAR,null,null,1.705,1.795
瑞典幣,SEK,null,null,2.91,3.03
紐元,NZD,18.01,18.86,18.34,18.64
泰幣,THB,0.8356,1.0256,0.946,0.992
菲國比索,PHP,0.4936,0.6256,null,null
印尼幣,IDR,0.00168,0.00238,null,null
歐元,EUR,33.27,34.61,33.785,34.385
韓元,KRW,0.02087,0.02477,null,null
越南盾,VND,0.00106,0.00147,null,null
馬來幣,MYR,6.414,7.939,null,null
人民幣,CNY,4.396,4.558,4.463,4.523



In [72]:
model = genai.GenerativeModel(
    "gemini-1.5-pro",
    system_instruction='''
    請將輸入的csv格式的字串,儲存為`現在日期.csv`
    ''',
    tools='code_execution'
)

response = model.generate_content(result_text)


In [75]:
generated_text = response.candidates[0].content 
print(generated_text)

parts {
  text: ""
}
parts {
  executable_code {
    language: PYTHON
    code: "\nimport csv\nfrom datetime import date\n\ncsv_string = \"\"\"幣別,幣別代碼,現金匯率(本行買入),現金匯率(本行賣出),即期匯率(本行買入),即期匯率(本行賣出)\n美金,USD,32.28,32.95,32.605,32.755\n港幣,HKD,4.04,4.244,4.161,4.231\n英鎊,GBP,39.46,41.58,40.355,40.985\n澳幣,AUD,20.03,20.81,20.245,20.59\n加拿大幣,CAD,22.19,23.1,22.52,22.85\n新加坡幣,SGD,23.6,24.51,24.07,24.29\n瑞士法郎,CHF,35.25,36.45,35.86,36.25\n日圓,JPY,0.2011,0.2139,0.2079,0.2129\n南非幣,ZAR,null,null,1.705,1.795\n瑞典幣,SEK,null,null,2.91,3.03\n紐元,NZD,18.01,18.86,18.34,18.64\n泰幣,THB,0.8356,1.0256,0.946,0.992\n菲國比索,PHP,0.4936,0.6256,null,null\n印尼幣,IDR,0.00168,0.00238,null,null\n歐元,EUR,33.27,34.61,33.785,34.385\n韓元,KRW,0.02087,0.02477,null,null\n越南盾,VND,0.00106,0.00147,null,null\n馬來幣,MYR,6.414,7.939,null,null\n人民幣,CNY,4.396,4.558,4.463,4.523\"\"\"\n\ntoday = date.today().strftime(\"%Y-%m-%d\")\nfilename = f\"{today}.csv\"\n\nwith open(filename, \'w\', newline=\'\', encoding=\'utf-8\') as csvfile:\n    writer = csv